In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVR
import math



In [23]:
# Import CVS
cropsDF = pd.read_csv('../data/agri/frontiers/Processed_Iowa+Cerro+Gordo_1960+2009_Annual+Crop.csv')

# Sort by year
cropsDF = cropsDF.sort_values(by='Year')
cropsDF.head()

# jimg Change Year to YEAR so it matches wx data
cropsDF['YEAR'] = cropsDF.Year
cropsDF.drop('Year', axis=1, inplace=True)

cropsDF.Value = cropsDF.Value.astype('float')

cropsDF

,Program,Period,Geo Level,State,Ag District,County,Commodity,Data Item,Domain Category,Value,YEAR
151,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,19.6,1940
150,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,51.8,1940
148,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,46.5,1941
149,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,14.9,1941
146,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,56.3,1942
...,...,...,...,...,...,...,...,...,...,...,...
4,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,163.1,2013
3,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,50.2,2014
2,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,168.3,2014
1,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,58.9,2015


In [25]:
# Filter data, look for 'BU/ACRE' and filter CORN & SOYBEANS
cornDF = cropsDF.loc[(cropsDF['Data Item'].str.contains('BU / ACRE')) & (cropsDF['Commodity'] == 'CORN')]
beansDF = cropsDF.loc[(cropsDF['Data Item'].str.contains('BU / ACRE')) & (cropsDF['Commodity'] == 'SOYBEANS')]

cornDF = cornDF[['YEAR', 'Value']]
beansDF = beansDF[['YEAR', 'Value']]

print('corn\n', cornDF.head())
print('beans\n', beansDF.head())

corn
      YEAR  Value
150  1940   51.8
148  1941   46.5
146  1942   56.3
144  1943   56.6
142  1944   49.8
beans
      YEAR  Value
151  1940   19.6
149  1941   14.9
147  1942   17.7
145  1943   18.8
143  1944   18.6


In [13]:
# GSP - Growing season precipitation
# GDD - Growing degree days
# GSTmax - Daily max temp avg
# GSTmin - Daily min temp avg
# frost - days < 0 degrees (C)
# summer - days > 25 degrees (C)


weatherDF = pd.read_csv('../data/wx/wx-frontier-agg.csv')
weatherDF.head()

# jimg removed averaging, all of these are individual features we need to keep

,INDEX,YEAR,GSP,GDD,GSTmax,GSTmin,frost,summer,HWI,CWI,dry,wet,PRCP95P
0,0,1941,3.869512,1555.1,25.865244,13.099390,2,94,11,7,13,5,4
1,1,1942,3.379878,1327.1,24.226220,11.957927,6,81,7,12,15,3,5
2,2,1943,3.396341,1303.8,23.991463,11.908537,7,80,6,7,10,4,2
3,3,1944,3.342331,1467.1,24.992073,12.899390,4,91,12,13,15,6,2
4,4,1945,3.471951,1130.0,23.137195,10.643293,6,66,5,13,19,3,5


In [26]:
# combine the weather features and the crop yield
combined = weatherDF.merge(cornDF, on='YEAR')
combined['corn'] = combined.Value
combined.drop('Value', axis=1, inplace=True)
combined = combined.merge(beansDF, on='YEAR')
combined['beans'] = combined.Value
combined.drop('Value', axis=1, inplace=True)
combined

,INDEX,YEAR,GSP,GDD,GSTmax,GSTmin,frost,summer,HWI,CWI,dry,wet,PRCP95P,corn,beans
0,0,1941,3.869512,1555.100,25.865244,13.099390,2,94,11,7,13,5,4,46.5,14.9
1,1,1942,3.379878,1327.100,24.226220,11.957927,6,81,7,12,15,3,5,56.3,17.7
2,2,1943,3.396341,1303.800,23.991463,11.908537,7,80,6,7,10,4,2,56.6,18.8
3,3,1944,3.342331,1467.100,24.992073,12.899390,4,91,12,13,15,6,2,49.8,18.6
4,4,1945,3.471951,1130.000,23.137195,10.643293,6,66,5,13,19,3,5,39.8,16.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,70,2011,2.235671,1428.125,24.988415,12.427744,2,92,9,10,14,4,0,169.3,47.7
71,71,2012,1.259451,1458.000,26.367683,11.412805,9,107,11,11,27,3,0,124.8,42.5
72,72,2013,3.569817,1394.850,24.716463,12.293902,4,82,7,10,21,15,0,163.1,40.3
73,73,2014,3.509146,1235.675,23.610366,11.458841,6,82,6,13,14,7,0,168.3,50.2


In [29]:
# from the paper: "The predictor and response variables were normalized prior 
# to their use by subtracting the mean and dividing by their standard deviation."

feats = ['YEAR', 'GSP', 'GDD', 'GSTmax', 'GSTmin', 'frost', 'summer' ,'HWI', 'CWI', 'dry' ,'wet', 'PRCP95P']
ys = ['corn', 'beans']

def norm(s):
    return (s - s.mean()) / s.std()

for f in feats + ys:
    combined[f] = norm(combined[f])
    
# jimg cross_val_score does not shuffle the data, so make sure we don't get
# test data that's a sequential set of years
combined = combined.sample(frac=1)

combined.head()

,INDEX,YEAR,GSP,GDD,GSTmax,GSTmin,frost,summer,HWI,CWI,dry,wet,PRCP95P,corn,beans
46,46,0.412948,-0.942434,1.425521,1.058177,1.621177,0.876163,1.056959,1.258906,-1.369767,1.770568,-0.253058,-1.325411,0.576761,0.920387
72,72,1.605910,0.296989,0.331767,0.101876,0.575132,-0.297270,-0.292013,-0.278846,-0.369125,1.130089,3.964579,-1.325411,1.263635,0.637082
4,4,-1.514144,0.182201,-1.716894,-1.473209,-1.680260,0.485019,-1.561634,-0.893946,0.631516,0.703103,-1.096586,1.564293,-1.550038,-1.505412
32,32,-0.229416,-0.087425,1.168906,0.549467,1.765316,-1.861846,0.263446,-0.586396,-1.036220,-1.004840,-0.674822,0.408411,-0.057627,-0.097740
14,14,-1.055312,-1.183452,1.783853,1.868828,1.282493,-0.297270,1.691770,3.719309,1.632157,1.557075,-0.253058,-0.169529,-1.403992,-1.558532


## Corn Data with weather data

In [32]:
# Separating the columns to split the data
X = combined[feats]
y = combined['corn']

# Splitting data into 80% training and 20% testing
# jimg cross_val_score will do this
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [33]:
# Using a SVR model, values
# were taken from the frontiers paper.
# jimg also need linear kernel
model = SVR(kernel='linear', C=0.1, epsilon=0.15)

In [34]:
# jimg cross_val_score will do this
# model.fit(X_train.to_numpy().reshape(-1, 1), y_train.to_numpy())

In [143]:
# Predict values using the testing data
# predictions = model.predict(X_test.to_numpy().reshape(-1, 1))

In [41]:
# Calculate the RMSE
# jimg 'neg_mean_squared_error' is -MSE so negate it
rmse = -cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5).mean()
r2 = cross_val_score(model, X, y, scoring="r2", cv=5).mean()

print('r2 %f rmse %f' %(r2, math.sqrt(rmse)))

r2 0.873287 rmse 0.353875


In [145]:
# This function is just to check the max, min, and avg of the rmse
# doing X iterations

# jimg we don't need this - cross_val_score doesn't shuffle so it will always return the same thing

#def max_min_rmse(model, iter):
#    max = 0
#    min = 9999
#    avg = 0
#    for i in range(iter):
#        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#        model.fit(X_train.to_numpy().reshape(-1, 1), y_train.to_numpy())
#        n = model.predict(X_test.to_numpy().reshape(-1, 1))
#        rmse = cross_val_score(SVR(), y_test.to_numpy().reshape(-1, 1), n, scoring="neg_mean_squared_error", cv=5)
#        rmse = np.absolute(rmse)
#        rmse = math.sqrt(rmse.mean())
#        avg += rmse

#        if min > rmse:
#            min = rmse
#        if max < rmse:
#            max = rmse
#    return round(min, 2), round(max, 2), round((avg/iter), 2)

In [38]:
# Running the function above
#iter = 1000
#min, max, avg = max_min_rmse(model, iter)

#print('Min RMSE: ', min, ' in ', iter, ' iterations')
#print('Max RMSE: ', max, ' in ', iter, ' iterations')
#print('Avg RMSE: ', avg, ' in ', iter, ' iterations')

## SOYBEANS with weather data

In [43]:
# Separating the columns to split the data
X = combined[feats]
y = combined['beans']


In [44]:
# Calculate the RMSE
# jimg 'neg_mean_squared_error' is -MSE so negate it
rmse = -cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5).mean()
r2 = cross_val_score(model, X, y, scoring="r2", cv=5).mean()

print('r2 %f rmse %f' %(r2, math.sqrt(rmse)))

r2 0.836171 rmse 0.394745
